# Finding Duplicate Reviews Via Semantic Search


In this walkthrough we will use vector embeddings to find duplicate or similar items in a movie review dataset.
The same approach can be used to group similar photos in your photo collection, automatically categorize data, etc.

## Outline

1. Get the data
2. Setup Lantern (on top of self-hosted postgres, or in Lantern Cloud)
3. Upload the data into Lantern
4. Generate embeddings (__automated in Lantern Cloud!__)
5. Create a vector index (__40x faster in Lantern Cloud!__)
6. Query the database to find similar reviews
    1. Brute Force - no vector index (takes ~1.5 hour)
    2. Vector Index + Code (takes ~20 minutes)
    3.  __Vector Index + SQL JOIN (takes ~40 seconds!)__
    
7. Bonus! Evaluate the quality of our approximate vector index
8. Bonus! Flag Identical Reviews


## 1. Get the data

In [ ]:
!python3 -m pip install psycopg2 datasets sentence_transformers tqdm ipywidgets pandas > /dev/null

We will use imdb movie review dataset from [huggingface](https://huggingface.co/datasets/imdb)

In [ ]:
import time
from datasets import load_dataset
import psycopg2
from psycopg2 import extras
from tqdm.notebook import tqdm
import pandas as pd

data = load_dataset("imdb", split="train")
data


## 2. Setup Lantern
You will need access to a Lantern database to follow through this tutorial. 

You can get one with 3 clicks at [Lantern Cloud](https://lantern.dev), or can set up Lantern on your own environment ([docs](https://docs.lantern.dev/get-started/install-from-binaries))

In [ ]:
# Connect to database
LANTERN_URL="PUT YOUR LANTERN URL HERE"
if not LANTERN_URL.startswith("postgres:"):
    LANTERN_URL=input("Please enter your Lantern URL:")
# Change the database URL to yours
def connect_db():
    return psycopg2.connect(LANTERN_URL)
global_conn = connect_db()
# idle_in_transaction_session_timeout

## 3. Upload the data into Lantern

Let's create a table for our movie review dataset with the following schema:
```sql
CREATE TABLE imdb_reviews (
    id SERIAL PRIMARY KEY, 
    imdb_id int NOT NULL UNIQUE, 
    review text, 
    positive_review bool)
```

Let's upload the data to our database so we can start running queries against it.
Note that we are using [`psycopg2.extras.execute_values`](https://www.psycopg.org/docs/extras.html#psycopg2.extras.execute_values) to handle batch uploading for us behind the scenes

In [ ]:
global_conn.autocommit = True
with global_conn.cursor() as cur:
    # set idle_in_transaction_session_timeout to 200ms to avoid excessive locking
    cur.execute("SET idle_in_transaction_session_timeout = 200")
    global_conn.commit()

In [ ]:
# Create table
def setup_table():
    with global_conn.cursor() as cur:
        #cur.execute("abort;CREATE EXTENSION IF NOT EXISTS lantern")
        #cur.execute("DROP TABLE IF EXISTS imdb_reviews")
        cur.execute("""
CREATE TABLE imdb_reviews (
  id SERIAL PRIMARY KEY,
  imdb_id int NOT NULL UNIQUE,
  review text,
  positive_review bool
);""")

In [ ]:
def insert_values(conn, values, batch_size=400, logging = True):
    start = time.time()
    with conn.cursor() as cur:
        batch_review = values["text"]
        batch_sentiment = values["label"]
        id_range = range(len(values["text"]))
        
        batch = list(zip(id_range,batch_review, batch_sentiment))
        batch = [(e[0], e[1], e[2] == 1) for e in batch]
        psycopg2.extras.execute_values(cur, f"INSERT INTO imdb_reviews (imdb_id,review, positive_review) VALUES %s;", batch,
                                      template=None, page_size=batch_size)
        conn.commit()

In [ ]:
setup_table()
insert_values(global_conn, data)

In [ ]:
## this is just a sanity check that imdb_ids in the postgres table correspond to our array indexes in this notebook
with global_conn.cursor() as cur:
    cur.execute("SELECT * from imdb_reviews where imdb_id = 1;")
    print(cur.fetchall()[0][2] == data["text"][1])

## 4. Generate embeddings
At this point we have all our data in our Lantern database. 
We can now go see some summary of our table in the Lantern dashboard.

More importantly, we can generate embeddings through various models, add them as additional columns to our table, and create vector indexes on them through the dashboard.
Lantern runs these operations on dedicated, workload-optimized servers, avoiding the extra load on the database instance.
This makes sure that your database will be available with its full capacity to answer your production queries, while very compute-heavy operations are carried out


<center><img alt="Generate embeddings in Lantern Cloud" src="../static/generate-embeddings-dashboard.gif" width="800"></center>

Once embedding generation and index creation finish successfully, we can see the additional columns on our table:

In [ ]:
with global_conn.cursor() as cur:
    form = "{:>26}" * 3
    cur.execute("SELECT column_name, data_type, is_nullable FROM information_schema.columns WHERE table_name = 'imdb_reviews';")
    print(form.format("column_name","data_type","is_nullable"))
    print()
    for r in cur.fetchall():
        # .join(["%s","%s","%s"])
        print(form.format(*r))

We will consider 3 approaches for solving the problem
1. No index, full scan of the table
2. Lantern index + python loop to aggregate results
3. Single JOIN query to get our answer

## 5. Create a vector index

We can again use the Lantern dashboard to create a vector index on the embedding column Lantern created for us.
Note that to create the vector index we could use the more familiar `CREATE INDEX` statement as below:
```python
with global_conn.cursor() as cur:
    cur.execute("""
    CREATE INDEX lantern_demo_idx ON lantern_demo 
    USING hnsw(vec dist_cos_ops) 
    WITH (m=32, ef_construction=128, dim=384, ef=64)""")
```

But vector index creation is an expensive operation - doing it inside the database will 
    - Take longer 
    - slow down database queries for the duration of index generation

Index creation done in Lantern Dashboard happens on a separate dedicated server. The resulting index is then copied over into our database and tied to postgres, as if it was created via `CREATE INDEX`.
This saves time and database resources! It also allows for faster iteration and index parameter tuning.



## 6. (A. and B.) Query the database

In [ ]:
# this is necessary for approaches (1) and (2) only, since embedding querying happens in python
all_embeds = None
with global_conn.cursor() as cur:
    cur.execute("SELECT imdb_id, review_embedding from imdb_reviews;")
    all_embeds = cur.fetchall() 

In [ ]:
def find_similar_foreach(all_embeds, use_index=True):
    THRESHHOLD=0.07
    # Load the next row from the dataset
    dist_calculation_format = "%s <-> review_embedding"
    if not use_index:
        dist_calculation_format = "cos_dist(%s, review_embedding)"

    with global_conn.cursor() as cur:

        for imdb_id, embedding in tqdm(all_embeds):
                
            cur.execute(f"SELECT cos_dist(%s, review_embedding) as dist, imdb_id from imdb_reviews order by {dist_calculation_format} limit 2;", 
                        (embedding,embedding))
            res = cur.fetchall()


            for r in res:
                dist, found_id = r
                if found_id == imdb_id:
                    continue
                if dist < THRESHHOLD:
                    print(f"found similar! (distance={dist})")
                    query_txt = data["text"][imdb_id]
                    print(f"Query({imdb_id}): {query_txt}")
                    found_txt = data["text"][found_id]
                    print(f"Found({found_id}): {found_txt}")

### Approach 6.A: Do not use the vector index: (WIll take ~ 1.5 hours)

In [ ]:

find_similar_foreach(all_embeds, False)

### Approach 6.B: Use the index but query it from python for each row: (WIll take ~ 25 minutes)

In [ ]:

find_similar_foreach(all_embeds, True)

### Approach 6.C: Vector Index + SQL JOIN (35seconds - 40x faster than above!)

The limitation of the above approach is that we are iterating over all movie reviews and issuing vector search operations. We can instead describe the full query to our database and have it return the final result - a list of review IDs and corresponding closest N review ids.

The query in the block below does exactly that!

In [ ]:
final_res = None
with global_conn.cursor() as cur:
    # increase work memory to make sure postgres has enough space to pin the relevant
    # table entries in memory
    cur.execute("SET work_mem='100MB'")
    # execute the actual clustering query!
    cur.execute("""
SELECT
  forall.imdb_id, 
  nearest_per_id.near_imdb_ids, nearest_per_id.imdb_dists
FROM
  (
    SELECT
      imdb_id, review_embedding
    FROM
      imdb_reviews
    LIMIT 25000
  ) AS forall
  JOIN LATERAL (
    SELECT
      ARRAY_AGG(imdb_id) AS near_imdb_ids, 
      ARRAY_AGG(imdb_dist) AS imdb_dists
    FROM
      (
        SELECT
          t2.imdb_id,
          forall.review_embedding <=> t2.review_embedding AS imdb_dist
        FROM
          imdb_reviews t2
        ORDER BY
          imdb_dist
        LIMIT
          5
      ) AS __unused_name
  ) nearest_per_id ON TRUE
ORDER BY
  forall.imdb_id;
""")
    final_res = cur.fetchall()

__What's going on in that query?__

There are two main subuqeries in the query above

1. Subquery forall:
This is the first building block of the query. It selects two pieces of information for each review in the dataset: the unique movie identifier (imdb_id) and the 'review embedding' (review_embedding). The review_embedding is a numerical representation of the review's content. This subquery is limited to the first 100,000 entries in the imdb_reviews_new1 table, indicating a focus on a specific portion of the dataset.


2. Lateral Join Subquery nearest_per_id:
The second building block is a more complex subquery that performs a lateral join. This means it takes each row from the forall subquery and finds the top 5 closest reviews to it based on the cosine distance between their embeddings. The cosine distance is a measure used to determine how similar two documents are in the context of natural language processing. This subquery aggregates the IDs (imdb_id) and distances (imdb_dist) of these closest reviews into arrays, essentially creating a list of most similar reviews for each review in the forall subset.

__Relation of the Outer Query to Building Block Queries:__
The outer query brings together these building blocks. It selects the imdb_id from the forall subquery and pairs it with the arrays of nearest imdb_ids and their corresponding distances (imdb_dists) from the nearest_per_id subquery. By joining these components, the query effectively maps each movie in the original subset to a list of movies with the most similar reviews, along with the degree of similarity. 
The final output is ordered by the imdb_id from the forall subset, providing an organized list of movies and their closest counterparts in terms of review content.

In [ ]:
very_similar = [r for r in final_res if 0.01 < r[2][1] and r[2][1] <= 0.03]

In [ ]:
pd.DataFrame(very_similar, columns=["imdb_id", "most_similar_imdb_ids", "distance"])

## Flag similar reviews
#### Below are some example pairs of reviews marked as similar according to our filtering above:

In [ ]:
print(data["text"][14384], "\n__VS__\n",data["text"][14396])

In [ ]:
print(data["text"][16401], "\n__VS__\n",data["text"][16408])


## Bonus! Evaluate the quality of our approximate vector index
Since we know for sure each vector must be closest to itself, we can use the clustering results to see how well our approximate index keeps this invariant. An exact vector index would always keep this invariant. HNSW sacrifices exactness for performance and it gives us 3 hyper-parameters to tune how close it tries to get to the exact index. Obviously, the more exact we make our approximate index, the slower it will become, so there is a tradeoff here.

We can again use Lantern dashboard to create indexes with different parameters and see which one results in fewer mistakes

In [ ]:
mistakes = [r for r in final_res if r[0] not in r[1]]
print("In %d (of %d reviews) the review in query was not considered close to itself by our index " % (len(mistakes), len(final_res)))

In [ ]:
mistakes = [r for r in final_res if r[0] not in r[1]]
print("In %d (of %d reviews) the review in query was not considered close to itself by our index " % (len(mistakes), len(final_res)))

## Bonus! Flag Identical Reviews
We can also use the query results from our index to find identical duplicate reviews in the dataset. To do this, we will search for vectors that are extremely close together and have different IMDB ids

In [ ]:
identical = [r for r in final_res if r[2][1] < 0.01]
identical[:10]

From the above, we see that there are many pairs of reviews that have very close to zero distance, This gives us very high confidence that the underlying reivews are identical Below are example identical rows, taken from the above

In [ ]:
data["text"][357]

In [ ]:
data["text"][10274]